# Syncing

## Git remote
The git remote command lets you create, view, and delete connections to other repositories. Remote connections are more like **bookmarks** rather than direct links into other repositories. Instead of providing real-time access to another repository, they serve as convenient names that can be used to reference a not-so-convenient URL.

To view the current remote configurations:
```bash
> git remote
origin
> git remote -v
origin	https://github.com/yuzaiusa/git_study_notes.git (fetch)
origin	https://github.com/yuzaiusa/git_study_notes.git (push)
```
The `-v` option will print the list of bookmarked repository names and additionally, the corresponding repository URL.

To manage a remote bookmark,
```bash
git remote add <name> <url>
git remote rm <name>
git remote rename <old_name> <new_name>
```

When you clone a repository with `git clone`, it automatically creates a remote connection called `origin` pointing back to the cloned repository. This is useful for developers creating a **local copy** of a central repository, since it provides an easy way to pull upstream changes or publish local commits. This behavior is also why most Git-based projects call their central repository origin.

Suppose we just pushed to or pulled from `origin`. `git log` will show that `HEAD`, `origin/master`, `origin/HEAD` are pointing to the same commit, the most recent one. If we made some changes and commit again, we will see that `HEAD` is one commit ahead of `origin/master` and `origin/HEAD`. `origin` remains to the be **latest fetched local copy** of `origin`.

```bash
git remote prune <name>
```
will **delete any local branches** for `<name>` that are not present on the remote repository.

To **inspect** a remote (the setting of the remote, not to fetch the remote's HEAD), run
```bash
git remote show <name>
```

## Git fetch

The git fetch command downloads commits, files, and refs from a remote repository into your local repo. Fetching is what you do when you want to see what everybody else has been working on. It’s similar to svn update in that it lets you see how the central history has progressed, but **it doesn’t force you to actually merge the changes** into your repository. Fetched content has to be explicitly checked out using the `git checkout` command. This makes fetching a **safe** way to review commits before integrating them with your local repository. On the other hand, `git pull` is the more **aggressive** alternative, it will download the remote content for the active local branch and immediately execute `git merge` to **create a merge commit** for the new remote content. If you have pending changes in progress this will cause **conflicts** and kickoff the merge conflict resolution flow.

To review local and remote branches, we can run
```bash
git branch
```
adn
```bash
git branch -r
```
, respectively.

To fetch a remote, run
```bash
git fetch <remote>
```
or
```bash
git fetch <remote> <branch>
```

To fetch all registed remotes and branches, run
```bash
git fetch --all
```

After fetching, we can switch to the checked out remote by
```bash
git checkout <remote>/<branch>
```
Very importantly, the fetched remote will be in a **detached HEAD** state. We are free to look around. But in order to make any commits, it is better to do
```bash
git checkout -b <new_branch_name>
```
in the detached HEAD branch.

Supoose we do a simple `git fetch`, which fetched the `origin/master`. To view the changes from the downloaded remote and the local HEAD, do
```bash
git log --oneline master..origin/master
```
If we feel comfortable, then
```bash
git merge origin/master
```

## Git push

The `git push` command is used to upload local repository content to a remote repository. It is essentially the same as running `git merge master` from inside the remote repository.

The basic usage is
```bash
git push <remote> <branch>
```

When the push results in **non-fast-forward** merge, meaning that your local branch is not based on the head of the remote. Git prevents you from overwriting the central repository’s history by refusing push requests when they result in a non-fast-forward merge. So, if the remote history has diverged from your history, you need to **pull** the remote branch and **merge** it into your local one, then try pushing again. However, you can do a **forced push** by
```bash
git push <remote> --force
```
**Do not use the `--force` flag unless you’re absolutely sure you know what you’re doing**. The only time you should ever need to force push is when you realize that the commits you just shared were not quite right and you fixed them with a **`git commit --amend`** or an **interactive rebase**. Once a commit is amended a `git push` will fail because Git will see the amended commit and the remote commit as **diverged content**. The `--force` option must be used to push an amended commit. However, you must be absolutely certain that none of your teammates have pulled those commits before using the `--force` option.

If you want to push **all branches** together, run
```bash
git push <remote> --al
```

Note that **tags are not automatically pushed** by default, or even by `--all`. To push tags, do
```bash
git push <remote> --tags
```

Since pushing messes with the remote branch structure, It is safest and most common to push to repositories that have been created with the `--bare` flag. **Bare repos** don’t have a working directory so a push will not alter any in progress working directory content.

One last usage of `git push` is to **delete a remote branch**.
```bash
git branch -D <name>
git push origin :<name>
```
Above commands will delete the remote branch by passing **a branch name prefixed with a colon** to `git push`.

## Git pull

The `git pull` command is used to fetch and download content from a remote repository and immediately update the local repository to match that content. The `git pull` command is actually a combination of two other commands, `git fetch` followed by `git merge`. In the first stage of operation `git pull` will execute a `git fetch` **scoped to the local branch that `HEAD` is pointed at**. Once the content is downloaded, `git pull` will enter a merge workflow. A **new merge commit** will be-created and **`HEAD` updated to point at the new commit**.

By default, `git pull` do merge first and then commit. It will change the local commit history and complicate the future push since the local changes appeared before the merged commit. A
```bash
git pull --rebase
```
will use the `git rebase` instead of `git merge` to integrate the remote branch with the local one. The `--rebase` option can be used to ensure a linear history by preventing unnecessary merge commits. Many developers prefer rebasing over merging, since it’s like saying, "I want to put my changes on top of what everybody else has done." In this sense, using `git pull` with the `--rebase` flag is even more like `svn update` than a plain `git pull`.

```bash
git pull <remote>
```
can pull from a remote other than the default.

```bash
git pull --no-commit <remote>
```
fetches the remote content (**does the merge?**) but does not create a new merge commit.